# Sim Network

In [117]:
import timeit
import swifter
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import combinations

from translate import translate_text

In [108]:
class cfg:
    TRAINING_DATA_PATH = './data/train.csv'

## Prepare data

In [113]:
df = pd.read_csv(cfg.TRAINING_DATA_PATH)
df = df[['id', 'name', 'latitude', 'longitude', 'city', 'state', 'zip', 'country', 'categories', 'point_of_interest']]
df = df[10000:21000]
df.head()

,id,name,latitude,longitude,city,state,zip,country,categories,point_of_interest
10000,E_0246d8074db266,LaFee Beach Club,39.583069,26.914058,Balikesir,Edremit,NaN,TR,"Nightclubs, Music Venues",P_91fe1a75b28452
10001,E_0246ea7e1057ab,Consultorio Odontológico,-25.356974,-57.505734,San Lorenzo,Central,NaN,PY,Dentist's Offices,P_d87a6bd4a758ea
10002,E_024708969f3193,Oak Knoll Park Baseball,33.820926,-118.033060,Cypress,CA,90630,US,Baseball Fields,P_b38f74f6ea56ff
10003,E_02471989168598,Mavi Köşe,36.588993,36.168899,İskenderun,Türkiye,31200,TR,"Fast Food Restaurants, Burger Joints",P_050caeb0086a49
10004,E_0247228a7f0483,Холмогорова 30,56.874012,53.217625,NaN,NaN,NaN,RU,Residential Buildings (Apartments / Condos),P_311d6daf9a16b1


In [114]:
df = df.fillna('NaN')

In [131]:
df['name'] = df['name'].swifter.apply(lambda x: translate_text(x.replace('.', '. ')))

AttributeError: 'Translator' object has no attribute 'raise_Exception'

In [ ]:
df['city'] = df['city'].swifter.apply(lambda x: translate_text(x.replace('.', '. ')))

In [ ]:
df['state'] = df['state'].swifter.apply(lambda x: translate_text(x.replace('.', '. ')))

In [ ]:
df['zip'] = df['zip'].swifter.apply(lambda x: translate_text(x.replace('.', '. ')))

In [ ]:
df['country'] = df['country'].swifter.apply(lambda x: translate_text(x.replace('.', '. ')))

In [ ]:
df['categories'] = df['categories'].swifter.apply(lambda x: translate_text(x.replace('.', '. ')))

In [132]:
pair1_cols = ['id1', 'name1', 'latitude1', 'longitude1', 'city1', 'state1', 'zip1', 'country1', 'categories1']
pair2_cols = ['id2', 'name2', 'latitude2', 'longitude2', 'city2', 'state2', 'zip2', 'country2', 'categories2']
pair_df = pd.DataFrame(columns=pair1_cols + pair2_cols)
new_pair = pair_df.copy(deep=True)
pair_df

,id1,name1,latitude1,longitude1,city1,state1,zip1,country1,categories1,id2,name2,latitude2,longitude2,city2,state2,zip2,country2,categories2


In [136]:
counter = 0
for poi, poi_df in tqdm(df.groupby('point_of_interest')):
    if len(poi_df) > 2:
        poi_df = poi_df.drop(columns=['point_of_interest'])
        new_pair = new_pair[0:0]
        match_combs = np.array(list(combinations(poi_df['id'], 2)))
        pair1 = poi_df[poi_df['id'].isin(match_combs[:, 0])]
        pair2 = poi_df[poi_df['id'].isin(match_combs[:, 1])]
        new_pair[pair1_cols] = pair1
        new_pair[pair2_cols] = pair2
        pair_df = pd.concat([pair_df, new_pair], ignore_index=True)
print(pair_df)

  4%|▍         | 486/10900 [00:00<00:13, 781.15it/s]


AttributeError: 'Translator' object has no attribute 'raise_Exception'

In [ ]:
from transformers import MarianTokenizer, MarianMTModel
import time

model_name = 'Helsinki-NLP/opus-mt-fr-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# function to get the chunks out of the entire data
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

for i, col in enumerate(source_cols):
    start = time.time()
    translations = []
    batch_no = 0
    # encode the source language text
    for source_text in chunks(data[col].tolist(), 500):
        batch_no += 1
        print('batch %d tokenization started' % batch_no)
        batch = tokenizer.encode(source_text, return_tensors='pt', padding=True)
        # predict the output token ids
        print('batch %d prediction started.' % batch_no)
        outputs = model.generate(**batch)
        print('batch %d decoding started.' % batch_no)
        decoded_output = tokenizer.decode(outputs, skip_special_tokens=True)
        translations.extend(decoded_output)
        print('batch %d completed' % batch_no)
    data[target_cols[i]] = translations
    end = time.time()
    print('%.2f hours taken for verbatim %s' % ((end - start)/3600, col))